In [ ]:
import numpy as np
from numpy.typing import NDArray
from typing import List, Optional, Tuple

In [48]:
def stationary_distribution(P: NDArray[np.float_], tol: float = 1e-12, maxiter: int = 10000) -> NDArray[np.float_]:
    """
    Compute the stationary distribution π such that πᵀ P = πᵀ.

    Uses power iteration on Pᵀ.

    Returns
    -------
    pi : 1D ndarray
        Stationary distribution.
    """
    n = P.shape[0]
    pi = np.ones(n) / n
    i = 0
    while True:
        pi_new = pi @ P
        if np.linalg.norm(pi_new - pi, 1) < tol:
            break
        pi = pi_new
        i+=1
        if i >= maxiter:
            val,vec = np.linalg.eig(P.T)
            return vec[:,0]/np.sum(vec[:,0])            
    return pi

def entropy_rate(P: np.ndarray, pi: Optional[np.ndarray] = None) -> float:
    """Shannon entropy rate *h = -∑_i π_i ∑_j P_ij log P_ij* in *bits* per step.

    Parameters
    ----------
    P : ndarray, shape (n, n)
        Row‑stochastic transition matrix.
    pi : Optional ndarray, shape (n,)
        Stationary distribution.  If *None* it is computed internally.
    base : float, default 2.0
        Logarithm base.  ``base=2`` → bits; ``np.e`` → nats.
    """
    if pi is None:
        pi = stationary_distribution(P)
    with np.errstate(divide="ignore", invalid="ignore"):
        logP = np.log(P) #/ np.log(base)
        logP[np.isneginf(logP)] = 0.0  # define 0·log0 = 0
        #h= -np.sum(np.sum(P * logP,axis=1)*pi)
        h = -(pi[:, None] * P * logP).sum()
    return float(h)
def time_reversed_transition_matrix(P: np.ndarray, pi: np.ndarray, eps=1e-15) -> np.ndarray:
    """
    Compute the time-reversed transition matrix from a row-stochastic matrix P and stationary distribution pi.

    Parameters
    ----------
    P : (N, N) ndarray
        Row-stochastic transition matrix P_{ij} = P(i → j)
    pi : (N,) ndarray
        Stationary distribution pi[i] > 0 and sum(pi) == 1
    eps : float
        Small number to avoid division by zero

    Returns
    -------
    P_rev : (N, N) ndarray
        Time-reversed transition matrix: P_{ij}(-tau)
    """
    pi = np.asarray(pi, dtype=float)
    P  = np.asarray(P, dtype=float)

    if P.shape[0] != P.shape[1] or P.shape[0] != pi.shape[0]:
        raise ValueError("Shape mismatch: P must be (N, N) and pi must be (N,)")
    P_rev = np.zeros(P.shape,dtype=float)
    for i in range(P.shape[0]):
        for j in range(P.shape[1]):
            if pi[i]!=0:
                P_rev[i,j] = pi[j]*P[j,i]/pi[i]

    return P_rev


In [49]:
# test entropy production
P = np.zeros((3,3),dtype=float)
P[0,1] = 0.5
P[0,2] = 0.5
P[1,0] = 1.
P[2,0] = 1.
for i in range(3):
   print(P[i].sum())
print(np.sum(P,axis=1))
print(P)
print(stationary_distribution(P))

1.0
1.0
1.0
[1. 1. 1.]
[[0.  0.5 0.5]
 [1.  0.  0. ]
 [1.  0.  0. ]]
[0.5  0.25 0.25]


In [50]:
print(entropy_rate(P))
print(-0.5*np.log(0.5))

0.3465735902799727
0.34657359027997264


In [35]:
val,vec = np.linalg.eig(P.T)
print(vec[:,0])
print(vec[:,0]@P)

[0.81649658 0.40824829 0.40824829]
[0.81649658 0.40824829 0.40824829]


In [53]:
pi = stationary_distribution(P)
with np.errstate(divide="ignore", invalid="ignore"):
    logP = np.log(P) #/ np.log(base)
    logP[np.isneginf(logP)] = 0.0  # define 0·log0 = 0
    #h= -np.sum(np.sum(P * logP,axis=1)*pi)
    print(P*logP)
    print(-(pi[:, None] * P * logP))


[[ 0.         -0.34657359 -0.34657359]
 [ 0.          0.          0.        ]
 [ 0.          0.          0.        ]]
[[-0.         0.1732868  0.1732868]
 [-0.        -0.        -0.       ]
 [-0.        -0.        -0.       ]]


In [56]:
A = np.array([1,2,3])
B = np.array([[1,2,3],[4,5,6],[7,8,9]])
print(A*B)

print(A[:,None]*B)

print(np.sum(B,axis=1))

[[ 1  4  9]
 [ 4 10 18]
 [ 7 16 27]]
[[ 1  2  3]
 [ 8 10 12]
 [21 24 27]]
[ 6 15 24]
